In [7]:
import pandas as pd
from pathlib import Path
import re

FOLDER = Path(r"C:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ISPU")

OUTPUT_FOLDER = Path(
    r"C:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ispu_named copy\input"
)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

PREFIX = "data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta"

for file in FOLDER.glob(f"{PREFIX}*.csv"):
    print(f"\n Processing: {file.name}")
    match = re.search(r"(20\d{2})", file.name)
    if not match:
        print(" Tahun tidak ditemukan di nama file, skip")
        continue

    file_year = int(match.group(1))
    print("➡️ Tahun file:", file_year)

    df = pd.read_csv(file)
    if "bulan" in df.columns:
        # 2024 & 2025
        df["bulan"] = df["bulan"].astype(str).str.zfill(2)
        df["tanggal"] = df["tanggal"].astype(str).str.zfill(2)

        df["tanggal"] = pd.to_datetime(
            df["bulan"].apply(lambda x: f"{file_year}-{x}")
            + "-"
            + df["tanggal"],
            errors="coerce"
        )
    else:
        # 2023
        df["tanggal"] = pd.to_datetime(df["tanggal"], errors="coerce")
    tahun_asli = df["tanggal"].dt.year
    mask_tanggal_mismatch = (tahun_asli.notna()) & (tahun_asli != file_year)

    if mask_tanggal_mismatch.any():
        print(" WARNING: Tahun di kolom tanggal tidak sesuai nama file!")
        print("   Contoh mismatch:")
        print(df.loc[mask_tanggal_mismatch, "tanggal"].head())
        print("   Jumlah mismatch:", mask_tanggal_mismatch.sum())

        # paksa ke tahun file
        df.loc[mask_tanggal_mismatch, "tanggal"] = (
            df.loc[mask_tanggal_mismatch, "tanggal"]
            .apply(lambda x: x.replace(year=file_year))
        )

    if "periode_data" in df.columns:
        tahun_periode = df["periode_data"].astype(str).str[:4]
        mask_periode_mismatch = tahun_periode != str(file_year)

        if mask_periode_mismatch.any():
            print(" WARNING: Tahun di periode_data tidak sesuai!")
            print("   Contoh periode_data mismatch:")
            print(df.loc[mask_periode_mismatch, "periode_data"].head())
            print("   Jumlah mismatch:", mask_periode_mismatch.sum())

        # sinkronkan dengan tanggal
        df["periode_data"] = df["tanggal"].dt.strftime("%Y%m")

    df["kode_stasiun"] = (
        df["stasiun"].str.extract(r"(DKI\d+)")
        .fillna(df["stasiun"].str.split().str[0])
    )

    df["id"] = (
        df["tanggal"].dt.strftime("%Y-%m-%d")
        + "_"
        + df["kode_stasiun"]
    )

    output_path = OUTPUT_FOLDER / f"{file_year}_id.csv"
    df.to_csv(output_path, index=False)

    print(f" Saved: {output_path}")



 Processing: data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv
➡️ Tahun file: 2023
   Contoh mismatch:
192   2022-12-01
193   2022-12-02
194   2022-12-03
195   2022-12-04
196   2022-12-05
Name: tanggal, dtype: datetime64[ns]
   Jumlah mismatch: 155
   Contoh periode_data mismatch:
192    202212
193    202212
194    202212
195    202212
196    202212
Name: periode_data, dtype: int64
   Jumlah mismatch: 155
 Saved: C:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ispu_named copy\input\2023_id.csv

 Processing: data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2024.csv
➡️ Tahun file: 2024
 Saved: C:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ispu_named copy\input\2024_id.csv

 Processing: data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2025.csv
➡️ Tahun file: 2025
 Saved: C:\Users\USER\Desktop\DATAVIDIA\penyisihan-datavidia-10\ispu_named copy\input\2025_id.csv
